In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
from sklearn import tree  # trying decision tree classifier
from sklearn import svm 

In [6]:
df = pd.read_csv("natural_proteins.csv")
df = df[df.chains ==1]  

# sheets = df.sheets[df.length <=1000]
# helices = df.helices[df.length <=1000]
# length = df.length[df.length <=1000]
# chains = df.chains[df.length <=1000]
# sequences = df.sequences[df.length <=1000]

names = df.names
sheets = df.sheets
helices = df.helices
chains = df.chains
sequences = df.sequences

converted = []
  
for i in sequences:
    # Converting string to list
    conv = i.strip('][').strip("'")#.split(', ')
    converted.append(conv)

sequences = converted

In [7]:
df = pd.DataFrame(list(zip(names, sequences, chains, helices, sheets)), columns = ["names", "sequences", "chains", "helices", "sheets"])
df.to_csv("1chain_proteins.csv")   # create csv

df = pd.read_csv("1chain_proteins.csv")  # read csv
df.head()


,Unnamed: 0,names,sequences,chains,helices,sheets
0,0,/Volumes/UCG Hard D./PDB Files/pdb/08/pdb208l.ent,KVFERCELARTLKRLGMDGYRGISLANWMCLAKWESGYNTRATNYN...,1,9,5
1,1,/Volumes/UCG Hard D./PDB Files/pdb/09/pdb209l.ent,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,1,8,2
2,2,/Volumes/UCG Hard D./PDB Files/pdb/13/pdb213l.ent,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,1,9,2
3,3,/Volumes/UCG Hard D./PDB Files/pdb/14/pdb214l.ent,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,1,9,2
4,4,/Volumes/UCG Hard D./PDB Files/pdb/18/pdb218l.ent,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,1,9,2


In [63]:
# Function that assigns numerical values to the acids in the sequences

import numpy as np
            
d3to1 = {'C': 0,'D': 1, 'S': 2, 'Q': 3, 'K': 4,
 'I': 5, 'P': 6, 'T': 7, 'F': 8, 'N': 9, 
 'G': 10, 'H': 11, 'L': 12, 'R': 13, 'W': 14, 
 'A': 15, 'V': 16, 'E': 17, 'Y': 18, 'M': 19, None: 20}

def sequence_enhancer(sequence, n):
    sequence_numeric_values = []
    
    for i in sequence:
        sequence_numeric_values.append(d3to1[i])
                
    if len(sequence) < n:
        for i in range(n-len(sequence)):
            sequence_numeric_values.append(20)
            
    return sequence_numeric_values

# Assigning the features and labels

X = []
y = sheets

lengths = []
for i in sequences:
    lengths.append(len(i))
max_len = max(lengths)
#counter = 0

for sequence in sequences:
    X.append(sequence_enhancer(sequence, max_len))
    #print(counter/8626*100, "% done")
    #counter= counter + 1

#print(len(X))


# Putting the features (positions) in the dataframe
X = pd.DataFrame(X)

#X = pd.concat([X, chains, length], ignore_index=True, axis=1)


In [64]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,1722,1723,1724,1725,1726,1727,1728,1729,1730,1731
0,4,16,8,17,13,0,17,12,15,13,...,20,20,20,20,20,20,20,20,20,20
1,19,9,5,8,17,19,12,13,5,1,...,20,20,20,20,20,20,20,20,20,20
2,19,9,5,8,17,19,12,13,5,1,...,20,20,20,20,20,20,20,20,20,20
3,19,9,5,8,17,19,12,13,5,1,...,20,20,20,20,20,20,20,20,20,20
4,19,9,5,8,17,19,12,13,5,1,...,20,20,20,20,20,20,20,20,20,20


In [65]:
# logistic regression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

clf = LogisticRegression(max_iter = 1000)     
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(accuracy_score(y_pred, y_test))


0.25301204819277107


/Users/maxschaffelder/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Checking the  logistic regression model using cross validation
cv_results = cross_validate(clf, X, y, cv=100)
print("Average accuracy is", np.average(cv_results["test_score"]))


In [ ]:
######################################################################################################

In [70]:
# decision tree

treee = tree.DecisionTreeClassifier()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 
treee = treee.fit(X_train, y_train)

y_pred = treee.predict(X_test)
#c_matrix = metrics.confusion_matrix(y_test, y_pred)
#print(c_matrix)
#print(" ")
print(accuracy_score(y_test, y_pred))  # accuracy for sheets

0.14457831325301204


In [ ]:
accuracy_list = []
for i in range(500):
    treee = tree.DecisionTreeClassifier()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 
    treee = treee.fit(X_train, y_train)

    y_pred = treee.predict(X_test)
    accuracy_list.append(accuracy_score(y_test, y_pred))
    
sum(accuracy_list)/len(accuracy_list)

In [ ]:
tree_results = cross_validate(treee, X, y, cv=50)
print(np.average(tree_results["test_score"]))

In [ ]:
#########################################################################################################

In [76]:
# Try support vector machine

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
clf = svm.SVC(C = 3)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.288
[[13  0  0  0  0  1  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 4  0 10  1  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 3  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 4  0  0  0  1  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 4  0  0  0  0  1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 3  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  2  1  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 6  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 4  0  0  0 

In [ ]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)
y_pred = svclassifier.predict(X_test)
print(accuracy_score(y_test, y_pred))